In [168]:
all_brands = set(['sonydigital', 'cyber', 'fujifilm', 'digital blue', 'bell howell', 'bell+howell', 'hello kitty', 'vistaquest', 'blackmagic', 'superheadz', 'hasselblad', 'hikivision', 'fuijifilm', 'panasonic', 'fugi film', 'fiji film', 'hikvision', 'polaroid', 'insignia', 'keychain', 'yourdeal', 'lowrance', 'logitech', 'fijifilm', 'bushnell', 'fujufilm', 'fugifilm', 'sylvania', 'olympul', 'lowepro', 'toshiba', 'olympus', 'samsung', 'finepix', 'fuifilm', 'samyang', 'aquapix', 'vivitar', 'neopine', 'minolta', 'easypix', 'sandisk', 'vivicam', 'philips', 'coleman', 'yashica', 'coolpix', 'emerson', 'sealife', 'crayola', 'intova', 'tamron', 'cannon', 'garmin', 'pentax', 'barbie', 'konica', 'mustek', 'aiptek', 'keedox', 'contax', 'konika', 'go pro', 'olymus', 'philip', 'wopson', 'pextax', 'wespro', 'disney', 'rollei', 'sakar', 'lytro', 'apple', 'lexar', 'kodak', 'vibe ', 'drift', 'vtech', 'vizio', 'kinon', 'haier', 'epson', 'gopro', 'croco', 'minox', 'nokia', 'dahua', 'sanyo', 'intel', 'kodax', 'ricoh', 'argus', 'lumix', 'nikon', 'absee', 'nicon', 'bmpcc', 'canon', 'casio', 'cobra', 'leica', 'sigma', 'sony', 'fuji', 'b h ', 'lego', 'benq', 'asus', 'pov', 'jvc', 'b+w', 'ion', 'lg ', 'dji', 'tvc', 'eos', 'ge ', 'vpc', 'dxg', 'svp', 'hp'])

In [2]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

w2v = Word2Vec.load("word2vec.model")

In [3]:
from os import listdir
from os.path import isfile, join
import json
import math

sentences = []

webs = [f for f in listdir('/home/sunji/EM_sigmod/2013_camera_specs')]
for w in webs:
    items = [f for f in listdir('/home/sunji/EM_sigmod/2013_camera_specs/'+w)]
    for it in items:
        with open('/home/sunji/EM_sigmod/2013_camera_specs/'+w+'/'+it, 'r') as f:
            data = json.load(f)
            sentence = ''
            for k, v in data.items():
                sentence += str(k)
                sentence += ' '
                if type(v) == list:
                    for vv in v:
                        sentence += str(vv)
                        sentence += ' '
                else:
                    sentence += str(v)
                    sentence += ' '
            sentences.append(sentence)

total_words_num = 0
            
word_freqs = {}
for s in sentences:
    for ss in s.split(' '):
        total_words_num += 1
        if ss.lower() in word_freqs:
            word_freqs[ss.lower()] += 1
        else:
            word_freqs[ss.lower()] = 1
for k in word_freqs.keys():
    word_freqs[k] /= float(total_words_num)

word_df = {}
for s in sentences:
    for ss in set(s.lower().split(' ')):
        if ss in word_df:
            word_df[ss] += 1
        else:
            word_df[ss] = 1
for k in word_df.keys():
    word_df[k] = math.log(len(sentences) / float(word_df[k]))
    
word_tfidf = {}
for k in word_df.keys():
    word_tfidf[k] = word_freqs[k] * word_df[k]
    

In [ ]:
sorted(word_tfidf.items(), key=lambda x:x[1], reverse=False)

In [4]:
import pandas as pd
structured_data = pd.read_csv('total_with_key_type.csv')
structured_data = structured_data.set_index('spec_id')

In [ ]:
candidate_pairs = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_quickstart.csv')

In [ ]:
len(structured_data[pd.isnull(structured_data['type'])])

In [ ]:
import pandas as pd
import swifter
products_labeled = pd.read_csv('/home/sunji/EM_sigmod/sigmod_large_labelled_dataset.csv')
# products_labeled['type'] = products_labeled['type'].fillna('')
# products_labeled['camera_title_overlap_freq'] = products_labeled['camera_title_overlap_freq'].fillna('')
# products_labeled['camera_title_unmatch_freq'] = products_labeled['camera_title_unmatch_freq'].fillna('')
def training_bloking(row):
    llll = structured_data.loc[row['left_spec_id']]['blocking_key']
    rrrr = structured_data.loc[row['right_spec_id']]['blocking_key']
    return llll == rrrr

products_labeled = products_labeled[products_labeled.swifter.apply(lambda x: training_bloking(x), axis=1)]


In [ ]:
import re
import editdistance

def feature_extraction(pairs_df):
    stopped_words = set(['digital camera', 'camera']) | all_brands

    camera_type_jaccard_dis = []
    camera_type_numeric_jaccard_dis = []
    camera_title_jaccard_dis = []
    camera_type_overlap_dis = []
    camera_type_numeric_overlap_dis = []
    camera_title_overlap_dis = []
    camera_title_overlap_freq = []
    camera_title_unmatch_freq = []
    camera_type_edit_dis = []
    camera_title_edit_dis = []
    camera_title_numeric_jaccard_dis = []
    camera_title_numeric_edit_dis = []
    
    cnt = 0
    for _, row in pairs_df.iterrows():
    #   camera_brand_dis
        if cnt % 100 == 0:
            print (cnt)
        cnt += 1
        llll = structured_data.loc[row['left_spec_id']]
        rrrr = structured_data.loc[row['right_spec_id']]

        lset = set(str(llll['type']).split(';'))
        rset = set(str(rrrr['type']).split(';'))
        
        max_len = max(len(str(llll['type'])), len(str(rrrr['type'])))
        camera_type_edit_dis.append(editdistance.eval(''.join(sorted(lset)), ''.join(sorted(rset))) / float(max_len))

        if len(lset) == 0 and len(rset) == 0:
            camera_type_jaccard_dis.append(0.0)
            camera_type_overlap_dis.append(0.0)
        else:
            camera_type_jaccard_dis.append(1.0 - len(lset&rset) / float(len(lset|rset)))
            camera_type_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

        lset = set(re.findall(r"\d+\.?\d*",str(llll['type'])))
        rset = set(re.findall(r"\d+\.?\d*",str(rrrr['type'])))

        if len(lset) == 0 and len(rset) == 0:
            camera_type_numeric_jaccard_dis.append(0.0)
            camera_type_numeric_overlap_dis.append(0.0)
        else:
            camera_type_numeric_jaccard_dis.append(1-len(lset&rset) / float(len(lset|rset)))
            camera_type_numeric_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

        lnumtitle = re.findall(r"\d+",str(llll['page_title']))
        rnumtitle = re.findall(r"\d+",str(rrrr['page_title']))
        
        lset = set(lnumtitle)
        rset = set(rnumtitle)
        
        if len(lset) == 0 and len(rset) == 0:
            camera_title_numeric_jaccard_dis.append(0.0)
            camera_title_numeric_edit_dis.append(0.0)
        else:
            camera_title_numeric_jaccard_dis.append(1.0 - len(lset&rset) / float(len(lset|rset)))
            max_len = max(len(lnumtitle), len(rnumtitle))
            camera_title_numeric_edit_dis.append(editdistance.eval(lnumtitle, rnumtitle) / float(max_len))
        
        ltitle = str(llll['page_title']).split(' ')
        rtitle = str(rrrr['page_title']).split(' ')
        lset = set(ltitle)
        rset = set(rtitle)

        if len(lset) == 0 and len(rset) == 0:
            camera_title_jaccard_dis.append(0.0)
            camera_title_overlap_dis.append(0.0)
            camera_title_edit_dis.append(0.0)
        else:
            camera_title_jaccard_dis.append(1.0-len(lset&rset) / float(len(lset|rset)))
            camera_title_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))
            max_len = max(len(ltitle), len(rtitle))
            camera_title_edit_dis.append(editdistance.eval(ltitle, rtitle) / float(max_len))
        overlap_freqs = []
        for w in lset&rset:
            if w in word_tfidf:
                overlap_freqs.append(str(word_tfidf[w]))
        camera_title_overlap_freq.append(';'.join(overlap_freqs))
        unmatch_freqs = []
        for w in (lset|rset)-(lset&rset):
            if w in word_tfidf:
                unmatch_freqs.append(str(word_tfidf[w]))
        camera_title_unmatch_freq.append(';'.join(unmatch_freqs))


    pairs_df['camera_type_jaccard_dis'] = camera_type_jaccard_dis
    pairs_df['camera_type_overlap_dis'] = camera_type_overlap_dis
    pairs_df['camera_type_edit_dis'] = camera_type_edit_dis
    pairs_df['camera_type_numeric_jaccard_dis'] = camera_type_numeric_jaccard_dis
    pairs_df['camera_type_numeric_overlap_dis'] = camera_type_numeric_overlap_dis
    pairs_df['camera_title_jaccard_dis'] = camera_title_jaccard_dis
    pairs_df['camera_title_overlap_dis'] = camera_title_overlap_dis
    pairs_df['camera_title_edit_dis'] = camera_title_edit_dis
    pairs_df['camera_title_numeric_jaccard_dis'] = camera_title_numeric_jaccard_dis
    pairs_df['camera_title_numeric_edit_dis'] = camera_title_numeric_edit_dis
    pairs_df['camera_title_overlap_freq'] = camera_title_overlap_freq
    pairs_df['camera_title_unmatch_freq'] = camera_title_unmatch_freq
   
    
        

In [ ]:
feature_extraction(products_labeled)

In [ ]:
products_labeled

In [ ]:
products_labeled.to_csv('products_labeled_with_distance.csv', index=False)

In [ ]:
import pandas as pd
products_labeled = pd.read_csv('products_labeled_with_distance.csv')
products_labeled['camera_title_overlap_freq'] = products_labeled['camera_title_overlap_freq'].fillna('')
products_labeled['camera_title_unmatch_freq'] = products_labeled['camera_title_unmatch_freq'].fillna('')

In [ ]:
products_labeled = products_labeled.drop_duplicates(subset=['camera_type_jaccard_dis',
       'camera_type_overlap_dis', 'camera_type_edit_dis',
       'camera_type_numeric_jaccard_dis', 'camera_type_numeric_overlap_dis',
       'camera_title_jaccard_dis', 'camera_title_overlap_dis',
       'camera_title_edit_dis', 'camera_title_overlap_freq',
       'camera_title_unmatch_freq', 'camera_title_numeric_jaccard_dis', 'camera_title_numeric_edit_dis'], keep='first')

In [ ]:
xxxx = products_labeled[products_labeled.duplicated()]
xxx = xxxx.sort_values(by=['camera_type_jaccard_dis',
       'camera_type_overlap_dis', 'camera_type_edit_dis',
       'camera_type_numeric_jaccard_dis', 'camera_type_numeric_overlap_dis',
       'camera_title_jaccard_dis', 'camera_title_overlap_dis',
       'camera_title_edit_dis', 'camera_title_overlap_freq',
       'camera_title_unmatch_freq', 'camera_title_numeric_jaccard_dis', 'camera_title_numeric_edit_dis'])


In [ ]:
len(xxx)

In [ ]:
len(products_labeled[products_labeled['label'] == 0])


In [ ]:
# train_features = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 3:11].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 3:11].values)
# train_overlap_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 11:12].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 11:12].values)
# train_unmatch_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 12:13].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 12:13].values)
# train_targets = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 2:3].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 2:3].values)

# test_features = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 3:11].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 3:11].values)
# test_overlap_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 11:12].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 11:12].values)
# test_unmatch_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 12:13].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 12:13].values)
# test_targets = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 2:3].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 2:3].values)

train_features = list(products_labeled[products_labeled['label']==1].iloc[:40000, 3:13].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 3:13].values)
train_overlap_freqs = list(products_labeled[products_labeled['label']==1].iloc[:40000, 13:14].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 13:14].values)
train_unmatch_freqs = list(products_labeled[products_labeled['label']==1].iloc[:40000, 14:15].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 14:15].values)
train_targets = list(products_labeled[products_labeled['label']==1].iloc[:40000, 2:3].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 2:3].values)

test_features = list(products_labeled[products_labeled['label']==1].iloc[40000:, 3:13].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 3:13].values)
test_overlap_freqs = list(products_labeled[products_labeled['label']==1].iloc[40000:, 13:14].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 13:14].values)
test_unmatch_freqs = list(products_labeled[products_labeled['label']==1].iloc[40000:, 14:15].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 14:15].values)
test_targets = list(products_labeled[products_labeled['label']==1].iloc[40000:, 2:3].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 2:3].values)


train_loader = torch.utils.data.DataLoader(
        ProductsDataset(train_features, train_targets, train_overlap_freqs, train_unmatch_freqs), batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        ProductsDataset(test_features, test_targets, test_overlap_freqs, test_unmatch_freqs), batch_size=1024, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import torch.utils.data

from torch.utils.data import DataLoader
import time

sequence_len = 1000
sequence_len_title = 20
class Dataset(object):
    """An abstract class representing a Dataset.

    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError


def generate_embeddings(words, seq_len):
    embedding = []
    for word in words:
        if word in w2v:
            embedding.append(w2v[word])
#     print ('length: ', len(embedding), len(words))
    if len(embedding) < seq_len:
        if len(embedding) > 0:
            mask = np.pad([[1] for _ in range(len(embedding))], ((0, seq_len-len(embedding)), (0,0)), 'constant')
            embedding = np.pad(embedding, ((0, seq_len-len(embedding)), (0,0)), 'constant')
        else:
            mask = np.pad([[0]], ((0, seq_len-1), (0,0)), 'constant')
            embedding = np.pad([[0 for _ in range(256)]], ((0, seq_len-1), (0,0)), 'constant')
    else:
        mask = np.array([[1] for _ in range(seq_len)])
        embedding = np.array(embedding[0:seq_len])
    return embedding, mask

class ProductsDataset(Dataset):
    """Dataset wrapping tensors.

    Each sample will be retrieved by indexing tensors along the first dimension.

    Arguments:
        *tensors (Tensor): tensors that have the same size of the first dimension.
    """

    def __init__(self, *tensors):
#         assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors

    def __getitem__(self, index):
        distances = self.tensors[0][index]
        targets = self.tensors[1][index]
        overlap_freqs = str(self.tensors[2][index][0])
        unmatch_freqs = str(self.tensors[3][index][0])
        overlap_len = 12
        unmatch_len = 30
        if len(overlap_freqs) == 0:
            overlap_freqs = '0.0'
        if len(unmatch_freqs) == 0:
            unmatch_freqs = '0.0'
        overlap_array = str(overlap_freqs).split(';')[0:overlap_len]
        unmatch_array = str(unmatch_freqs).split(';')[0:unmatch_len]
        
        overlap = np.pad([float(x) for x in overlap_array], (0, overlap_len-len(overlap_array)), 'constant')
        unmatch = np.pad([float(x) for x in unmatch_array], (0, unmatch_len-len(unmatch_array)), 'constant')
        return tuple([torch.FloatTensor(distances), torch.FloatTensor(targets), torch.FloatTensor(overlap), torch.FloatTensor(unmatch)])

    def __len__(self):
        return len(self.tensors[0]) 

use_cuda = torch.cuda.is_available()

#     torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

# train_loader = torch.utils.data.DataLoader(
#         ProductsDataset(train_features, train_overlap_freqs, train_unmatch_freqs, train_targets), batch_size=1024, shuffle=True)
# test_loader = torch.utils.data.DataLoader(
#         ProductsDataset(test_features, test_overlap_freqs, test_unmatch_freqs, test_targets), batch_size=1024, shuffle=True)



# train_loader2 = torch.utils.data.DataLoader(
#         ProductsDataset2(), batch_size=128, shuffle=True)
# test_loader2 = torch.utils.data.DataLoader(
#         ProductsDataset2(), batch_size=128, shuffle=True)


In [ ]:

import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Model(nn.Module):
    
    def __init__(self, input_size):
        super(Model, self).__init__()
        
        self.linear1 = nn.Linear(input_size, 256)
        self.linear2 = nn.Linear(256, 128)
        
    def forward(self, features):
        hid = F.relu(self.linear1(features))
        hid = F.relu(self.linear2(hid))
        return hid
    
class Output(nn.Module):
    
    def __init__(self, input_len):
        super(Output, self).__init__()
        self.linear1 = nn.Linear(input_len, 256)
        self.linear3 = nn.Linear(256, 1)
        
    def forward(self, embeddings):
        inputs = torch.cat(embeddings, dim=1)
        hid = F.relu(self.linear1(inputs))
        hid = F.sigmoid(self.linear3(hid))
        return hid

def l1_loss(estimates, targets):
#     targets = targets.unsqueeze(1)
#     print (torch.cat((estimates, targets), 1))
    return F.mse_loss(estimates, targets)
#     return (-targets * (torch.log(estimates + 0.0001)) - (1.0 - targets) * (torch.log(1.0 - estimates + 0.0001))).mean()

def print_loss(estimates, targets):
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for i in range(len(estimates)):
        if estimates[i][0] >= 0.5 and targets[i] == 1:
            true_positive += 1
        elif estimates[i][0] < 0.5 and targets[i] == 1:
            true_negative += 1
        elif estimates[i][0] < 0.5 and targets[i] == 0:
            false_negative += 1
        else:
            false_positive += 1
    if true_positive + false_positive == 0:
        precision = 1
    else:
        precision = float(true_positive) / (true_positive + false_positive)
    if true_positive + true_negative == 0:
        recall = 1
    else:
        recall = float(true_positive) / (true_positive + true_negative)
    return precision, recall
        
# strctured_model = Model()
# unstructured_model = bilstm_attn(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 1000)
# title_model = bilstm_attn(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 20)
# output_model = Output()

# new_structured_model = Model()
# new_title_model = Attention(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 20)
# new_output_model = Output()

# torch.manual_seed(1)
# distance_model = Model(10)
# overlap_model = Model(12)
# unmatch_model = Model(30)
# output_model = Output(384)

# opt = optim.Adam([{'params':output_model.parameters()},{'params':distance_model.parameters()},{'params':overlap_model.parameters()},{'params':unmatch_model.parameters()}], lr=0.001)

for e in range(20):
    distance_model.train()
    overlap_model.train()
    unmatch_model.train()
    output_model.train()
    for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(train_loader):
        distance = Variable(distances)
        target = Variable(targets)
        overlap = Variable(overlaps)
        unmatch = Variable(unmatches)
        
        opt.zero_grad()
        
        distance_embedding = distance_model(distance)
        overlap_embedding = overlap_model(overlap)
        unmatch_embedding = unmatch_model(unmatch)
        
        estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
        loss = l1_loss(estimates, targets)
        print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
        loss.backward()
        opt.step()
        
    distance_model.eval()
    overlap_model.eval()
    unmatch_model.eval()
    output_model.eval()
    for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(test_loader):
        distance = Variable(distances)
        target = Variable(targets)
        overlap = Variable(overlaps)
        unmatch = Variable(unmatches)
        
        distance_embedding = distance_model(distance)
        overlap_embedding = overlap_model(overlap)
        unmatch_embedding = unmatch_model(unmatch)
        
        estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
        loss = l1_loss(estimates, targets)
        
        prec, rec = print_loss(estimates, targets)
        
        print('Testing: Batch {}, Loss {}, Precision {}, Recall {}'.format(batch_idx, loss.item(), prec, rec))

In [ ]:
test_pairs = candidate_pairs.sample(n=10000, random_state=7)
feature_extraction(test_pairs)


In [ ]:
data_loader = torch.utils.data.DataLoader(
        ProductsDataset(candidate_pairs.iloc[:, 4:14].values, candidate_pairs.iloc[:, 5:6].values, candidate_pairs.iloc[:, 14:15].values,candidate_pairs.iloc[:, 15:16].values), batch_size=1000, shuffle=False)

distance_model.eval()
overlap_model.eval()
unmatch_model.eval()
output_model.eval()

scores = []
matches = 0

for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(data_loader):
    distance = Variable(distances)
    target = Variable(targets)
    overlap = Variable(overlaps)
    unmatch = Variable(unmatches)

    distance_embedding = distance_model(distance)
    overlap_embedding = overlap_model(overlap)
    unmatch_embedding = unmatch_model(unmatch)
    
    estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
    
    for i in range(estimates.shape[0]):
        predict = estimates[i].item()
        if predict >= 0.5:
            matches += 1
        scores.append(predict)
    print ('Matching Pairs: {}, Total Pairs: {}'.format(matches, batch_idx * 1000))
    
# ['scores'] = scores
    

In [ ]:
candidate_pairs['scores'] = scores

In [ ]:
candidate_pairs[candidate_pairs['scores'] >= 0.6][['left_spec_id', 'right_spec_id']].to_csv('submit_v8.csv', index=False)


In [ ]:
candidate_pairs[candidate_pairs['scores'] >= 0.999][['left_spec_id', 'right_spec_id']].to_csv('submit_v10.csv', index=False)


In [ ]:
# torch.save(strctured_model.state_dict(), 'structured.model')
# torch.save(title_model.state_dict(), 'title.model')
# torch.save(output_model.state_dict(), 'output.model')

# torch.save(distances_model.state_dict(), 'distances_model.model')
# torch.save(freqs_model.state_dict(), 'freqs_model.model')

torch.save(distance_model.state_dict(), 'distance_model.model')
torch.save(overlap_model.state_dict(), 'overlap_model.model')
torch.save(unmatch_model.state_dict(), 'unmatch_model.model')
torch.save(output_model.state_dict(), 'output_model.model')

In [ ]:
distance_model = Model(10)
overlap_model = Model(12)
unmatch_model = Model(30)
output_model = Output(384)

distance_model.load_state_dict(torch.load('distance_model.model'))
overlap_model.load_state_dict(torch.load('overlap_model.model'))
unmatch_model.load_state_dict(torch.load('unmatch_model.model'))
output_model.load_state_dict(torch.load('output_model.model'))



In [160]:
import pandas as pd
candidates = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_quickstart.csv')
# structured_data = pd.read_csv('total_with_key_type.csv')

In [787]:
xxxx = pd.read_csv('/home/sunji/EM_sigmod/sigmod_large_labelled_dataset.csv')
# xxxx = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/output/submission.csv')
# xxxx = pd.read_csv('/home/sunji/EM_sigmod/miss.csv')
# xxxx = pd.read_csv('/home/sunji/EM_sigmod/submit_v26_97_98_97.csv')
# xxxx.columns = ['left_spec_id', 'right_spec_id', 'left_spec_title', 'right_spec_title']

In [790]:
xxxx = candidates.join(structured_data[['type', 'type_number', 'blocking_key', 'page_title', 'version']], on='left_spec_id', how='inner').join(structured_data[['type', 'type_number', 'blocking_key', 'page_title', 'version']], on='right_spec_id', how='inner', lsuffix='_left', rsuffix='_right')


In [789]:
xxxx.to_csv('/home/sunji/EM_sigmod/quickstart_package/label_pairs_with_key_type.csv', index=False)

In [574]:
xxxx.to_csv('/home/sunji/EM_sigmod/pairs_with_key_type.csv', index=False)

In [791]:
xxxx.to_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_with_key_type.csv', index=False)

In [36]:
structured_data = structured_data.set_index('spec_id')

In [779]:
def findlong(x):
    return len(str(x['type']).split(';')) == 1

def findlong2(x):
    return len(str(x['type'])) == 0
# xxxx[(xxxx['left_spec_id'] == 'www.eglobalcentral.co.uk//713') & (xxxx['right_spec_id'] == 'www.eglobalcentral.co.uk//637')]
x = structured_data[structured_data.apply(lambda row: findlong2(row), axis=1)]
x[x['source'] != 'www.alibaba.com'].sample(50)


,source,spec_number,page_title,blocking_key,type,camera_brand,camera_model,camera_color,camera_resolution,camera_sensitivity,camera_stablization,camera_battery,camera_mpn,camera_title,type_number,version
spec_id,,,,,,,,,,,,,,,,
www.eglobalcentral.co.uk//70,www.eglobalcentral.co.uk,70,canon ef 85mm f/1.8 usm lenses,cannon,,canon,NaN,unknown,0.000000,0.00000,unknown,unknown,unknown,Canon EF 85mm f/1.8 USM Lenses,,
www.ebay.com//54182,www.ebay.com,54182,samsung purple touchscreen camera | ebay,samsung,,samsung,NaN,unknown,0.406667,0.00000,unknown,unknown,unknown,Samsung Purple Touchscreen Camera | eBay,,
www.ebay.com//56151,www.ebay.com,56151,mini vivitar digital camera | ebay,vivitar,,vivitar,mini;digital camera,unknown,0.000000,0.00000,unknown,unknown,unknown,Mini Vivitar Digital Camera | eBay,,
www.eglobalcentral.co.uk//463,www.eglobalcentral.co.uk,463,nikon af-s nikkor 14-24mm f/2.8g ed lenses,nikon,,nikon,NaN,unknown,0.000000,0.00000,unknown,unknown,unknown,Nikon AF-S NIKKOR 14-24mm f/2.8G ED Lenses,,
www.ebay.com//46724,www.ebay.com,46724,brand new hasselblad stellar edititon white | ...,hasselblad,,hasselblad,NaN,unknown,0.000000,0.00000,unknown,unknown,unknown,Brand New Hasselblad Stellar Edititon White | ...,,
www.ebay.com//59820,www.ebay.com,59820,vivitar vivicam 14mp itwist digital camera | ebay,vivitar,,vivitar,digital camera,unknown,0.466667,0.00000,unknown,unknown,unknown,Vivitar ViviCam 14MP Itwist Digital Camera | eBay,,
www.eglobalcentral.co.uk//295,www.eglobalcentral.co.uk,295,tamron af 18-200mm f/3.5-6.3 xr di-ii ld asphe...,tamron,,tamron;nikon,NaN,unknown,0.000000,0.00000,unknown,unknown,unknown,Tamron AF 18-200mm f/3.5-6.3 XR Di-II LD Asphe...,,
www.eglobalcentral.co.uk//494,www.eglobalcentral.co.uk,494,sigma art 18-35mm f1.8 dc hsm for canon mount,cannon,,canon;sigma,NaN,unknown,0.000000,0.00000,unknown,unknown,unknown,Sigma ART 18-35mm F1.8 DC HSM for Canon Mount,,
www.ebay.com//47780,www.ebay.com,47780,disney princess digital camera w preview scree...,disney,,disney,digital camera,unknown,0.070000,0.00000,unknown,unknown,unknown,Disney Princess Digital Camera w Preview Scree...,,


In [785]:
import re
stopped_words = set(['cam','ixus','pro','rebel','shot','cyber','catalog','stylus','mark','edition','elph','','digital camera', 'camera','wide','model','sport','network','action','full','ir','mini','metal','digital camera', 'indoor', 'canon eos', 'compact', 'digital', 'digital rebel', 'case', 'lcd', 'panasonic lumix', 'slr', 'cybershot', 'hd', '3x', '4x', '5x', '3gp', '']) | all_brands
def is_type(words, idx, current_num):
#     if len(words) > 3:
#         return True
#     print (words[idx], current_num)
    exist_number_match = re.findall(r'[0-9]+', words[idx])
    only_number_match = re.findall(r'^[0-9]+$', words[idx])
    possible_mpn_match = re.findall(r'\d{4}', words[idx])
    letter_match = re.findall(r'[a-z]+', words[idx])
    stop_match = re.findall(r'["$\']+', words[idx])
    if 'comparison' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
#     if 'buy' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
#         return False
    if 'kit' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if '&' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if '+' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'w/' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'w' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'with' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if len(words[idx]) > 20:
        return False
    if not exist_number_match and words[idx] not in ['i','ii','iii','vi','iv', 'iis', 'slv', 'df', 'f1', 'm', 'x', 'nx', 'xs', 'k-x', 'k-r', 'xs-pro', 'v', 'gr', 'px']: # No number, cannot be type
        return False
    if only_number_match and (len(set(words[0:idx]) & all_brands) == 0 or len(words[idx]) > 4):
        return False
    if not letter_match:
        if stop_match: # money, size etc.
            return False
        if idx < len(words)-1 and (words[idx+1] in ['batteries', 'gb', 'mm', 'mp', 'inch', 'megapixel', 'mega', 'pack']):
            return False
#         if not possible_mpn_match:
        if len(set(words[max(0, idx-2):idx]) & all_brands) == 0 and words[idx-1] not in ['rebel', 'finepix'] or (words[idx-2] == 'eos' and words[idx-1] != 'rebel') or words[idx-2] == 'finepix': # no adjacent brand
            if current_num >= 1:
                return False
            if idx < len(words)-1 and (words[idx+1].endswith('mm') or words[idx+1].endswith('mp') or words[idx+1].endswith('inch') or words[idx+1].endswith('megapixel') or words[idx+1].endswith('mega')):
                return False
            if only_number_match and idx < len(words)-2 and re.findall(r'^[0-9]+$', words[idx+1]):
                if float(words[idx+1]) < 10 and (words[idx+2] in ['mp', 'inch', 'mega', 'megapixel']): # still a pixel, missing point
                    return False
                if float(words[idx]) < 300 and float(words[idx+1]) < 1000 and words[idx+2] in ['mm', 'lens']: # still a len, missing point
                    return False
            if re.findall(r'[0-9]+\-[0-9]+', words[idx]):
                return False
        else:
            if only_number_match and idx < len(words)-2 and re.findall(r'^[0-9]+$', words[idx+1]):
                if float(words[idx]) >= 10 and float(words[idx+1]) < 10 and (words[idx+2] in ['mp', 'inch', 'mega', 'megapixel']): # still a pixel, missing point
                    return False
                if float(words[idx]) < 300 and float(words[idx+1]) < 1000 and words[idx+2] in ['mm', 'lens']: # still a len, missing point
                    return False
            if current_num >= 1 and idx < len(words)-1 and (words[idx+1].endswith('mm') or words[idx+1].endswith('mp') or words[idx+1].endswith('inch') or words[idx+1].endswith('megapixel') or words[idx+1].endswith('mega')):
                return False
        if words[idx] in ['2015', '2014', '2013', '2012', '2011']:
            return False
    else:
#         print (words[idx])
        if words[idx] in ['i', 'ii', 'iii', 'iv'] and 'camera' not in ''.join(words[max(0,idx-2):idx]) and (idx > 0 and words[idx-1] not in ['mark', 'mk']) and ('af-s' in words[0:idx] or (idx < len(words)-1 and words[idx+1] == 'lens') or len(set(words[max(0, idx-4):idx]) & all_brands) == 0):
            return False
        if words[idx] in ['iis', 'm', 'x', 'df', 'f1', 'gr', 'px', 'slv', 'xs', 'k-x', 'k-r', 'xs-pro', 'v'] and (len(set(words[max(0, idx-4):idx]) & all_brands) == 0 or current_num > 0):
            return False
        if words[idx] in ['nx'] and words[idx+1] != 'mini':
            return False
        if (re.findall(r'\d*\.?\d*\-?\d*\.?\d*mm$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mp$', words[idx]) or
           re.findall(r'^\d+\.?\d*\-?\d*\.?\d*m$', words[idx]) or
           re.findall(r'^\d+\.?\d*\-?\d*\.?\d*cm$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megpixel$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*cmos$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pcs$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pc$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*inch$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*gb$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*fps$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixel$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mega$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixels$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*year$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*yr$', words[idx]) or
           re.findall(r'^s\d+\.?\d*\-\d+\.?\d*$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g?hz$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*lens$', words[idx]) or
           re.findall(r'ip\d*$', words[idx])):
#             print (words[idx])
            return False
        if words[idx] in ['1080p', '960p', '720p', '360p', 'h.264', 'p2p']:
            return False
        if words[idx].startswith('sel') or words[idx].startswith('sal'):
            return False
        if re.findall(r'[0-9]+\-[0-9]+\/[0-9]+-[0-9]+$', words[idx]):
            return False
        if re.findall(r'^f\/[0-9]+\.?[0-9]*', words[idx]):
            return False
        if re.findall(r'^f\/?[0-9]+\.?[0-9]*$', words[idx]) and len(set(words[max(0, idx-2):idx]) & all_brands) == 0:
            return False
        if re.findall(r'^f\/?[0-9]+\.?[0-9]*\-[0-9]+\.?[0-9]*', words[idx]):
            return False
        if stop_match:
            return False
        if re.findall(r'^[0-9]+\.?[0-9]*x$', words[idx]) and len(set(words[max(0, idx-2):idx]) & all_brands) == 0:
            return False
        
    return True

def verify_model(word, current_size):
#     print (word)
    if re.findall(r'\d+', word) and current_size > 0:
#         print ('here')
        return False
    if (re.findall(r'\d*\.?\d*\-?\d*\.?\d*mm$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mp$', word) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*m$', word) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*cm$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megpixel$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mega$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*cmos$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pcs$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pc$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*inch$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*gb$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*fps$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixel$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixels$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*year$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*yr$', word) or
           re.findall(r'^s\d+\.?\d*\-\d+\.?\d*$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g?hz$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*lens$', word) or
           re.findall(r'ip\d*$', word)):
#         print ('here2')
        return False
    if word in ['1080p', '720p', '360p', 'h.264', 'p2p']:
        return False
    if word.startswith('sel') or word.startswith('sal'):
        return False
    if re.findall(r'[0-9]+\-[0-9]+\/[0-9]+-[0-9]+$', word):
        return False
    if re.findall(r'^f\/[0-9]+\.?[0-9]*', word):
        return False
    if re.findall(r'^f\/?[0-9]+\.?[0-9]*$', word):
        return False
    if re.findall(r'^f\/?[0-9]+\.?[0-9]*\-[0-9]+\.?[0-9]*', word):
        return False
    if re.findall(r'^[0-9]+\.?[0-9]*x$', word):
        return False
    return True

def tfidf(x):
#     print (x)
    if x in word_tfidf:
        return word_tfidf[x]
    else:
        return 0.0

def extract_type(row):
    words = re.split(r'[, ()~]', str(row['page_title']))
    brand = str(row['blocking_key'])
    words = [x.strip(',;-()') for x in words]
    types = []
    version = None
    generations = {'i':'1','ii':'2','iii':'3','iv':'4'}
    generations2 = {'i':'one','ii':'two','iii':'three','iv':'four'}
    keep_num = 2
#     print (words)
    for idx, w in enumerate(words):
#         print (w)
        if is_type(words, idx, len(re.findall(r'\d+', ''.join(types)))):
            if w in ['i','ii','iii','iv']:
                if version is None:
                    version = generations[w]
                if len(types) == 0:
                    types.append(generations2[w])
#             if re.findall(r'\d+', w) or w in ['i','ii','iii','vi','iv']:
#             elif w in ['i','ii','iii','iv']:
#                 types.append(generations2[w])
            elif w == 'm' and idx > 1 and words[idx-1] == 'eos':
                types.append('eosm')
            elif w == 'x' and idx > 1 and words[idx-1] == 'k':
                types.append('kx')
            else:
                if re.findall(r'^\d+$', w):
#                     print ('here: ', w)
                    if idx > 0 and words[idx-1] in all_brands:
                        types.append(w)
                    elif idx > 1 and words[idx-2] in all_brands:
                        if not re.findall(r'\d+', words[idx-1]) and words[idx-1] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-1] + '-' + w)
                    elif idx > 2 and words[idx-3] in all_brands:
                        if not re.findall(r'\d+', words[idx-1]) and words[idx-1] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-1] + '-' + w)
                        if not re.findall(r'\d+', words[idx-2]) and words[idx-2] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-2] + '-' + w)
                        keep_num += 1
                    elif idx > 3 and words[idx-4] in all_brands:
                        if not re.findall(r'\d+', words[idx-1]) and words[idx-1] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-1] + '-' + w)
                        if not re.findall(r'\d+', words[idx-2]) and words[idx-2] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-2] + '-' + w)
                        if not re.findall(r'\d+', words[idx-3]) and words[idx-3] not in ['camera', 'digital', 'slr', 'series']:
                            types.append(words[idx-3] + '-' + w)
                        keep_num += 2
                    else:
                        types.append(w)
                else:
                    types.append(w)
#             types = list(set(types) - (stopped_words - all_brands))
#     print (types)
    if len(types) == 0 and row['camera_model'] != 'unknown' and row['camera_model'] != 'nan' and not pd.isnull(row['camera_model']):
        models = re.split(r'[; ]', str(row['camera_model']))
        mmmmms = []
        for m in models:
            if verify_model(m, len(types)):
                mmmmms.append(m.strip('"\''))
        models = set(mmmmms) - stopped_words
#         models = sorted(re.split(r'[; ]', str(row['camera_model'])), key=lambda x: tfidf(x), reverse=False)
#         models = models[0:min(len(models), 2)]
        models = sorted(models, key=lambda x: tfidf(x), reverse=False)
        if len(models) > 0:
            types = set(types) | set(models[0:1])
    if len(types) == 0 and row['camera_mpn'] != 'unknown':
        types.append(str(row['camera_mpn']))
    types = set([''.join(t.split('-')) for t in types]) - stopped_words
    types = sorted(types, key=lambda x: tfidf(x), reverse=False)
#     types = types[0:min(len(types), 5)]
#     has_number = []
#     for i in range(len(types)):
#         if re.findall(r'\d+', types[i]):
#             has_number.append(i)
#     if len(has_number) > 0:
#         typess = []
#         for ii in has_number:
#             typess.append(types[ii])
#         types = typess
    types = types[0:min(len(types), keep_num)]
    for idx, t in enumerate(types):
        if 'rx100' in t and t.endswith('b'):
            types[idx] = types[idx][0:-1]
        elif t.endswith('/b'):
            types[idx] = types[idx][0:-2]
        if version is None:
            if t.endswith('iii'):
                version = '3'
#                 types[idx] = types[idx][0:-3]
            elif t.endswith('ii'):
                version = '2'
#                 types[idx] = types[idx][0:-2]
            elif t.endswith('m3'):
                version = '3'
#                 types[idx] = types[idx][0:-2]
            elif t.endswith('m2'):
                version = '2'
#                 types[idx] = types[idx][0:-2]
    if version is None:
        version = ''
    return ';'.join(types), version

def extract_type_number(row):
    all_types = str(row['type']).split(';')
    numbers = []
    for t in all_types:
        num = re.findall(r'\d+', str(t))
        if len(num) > 0:
            numbers.append(''.join(num))
    return ';'.join(numbers)

def some_extra_rule(row):
    all_types = str(row['type']).split(';')
    for i in range(len(all_types)):
        if all_types[i] == 'g1' and str(row['blocking_key']) == 'cannon':
            all_types[i] = 'g1x'
        elif all_types[i] == '7k' and str(row['blocking_key']) == 'sony':
            if 'nex' in row['page_title']:
                all_types[i] = 'nex7'
            elif 'ilce' in row['page_title']:
                all_types[i] = 'a7'
            elif 'alpha' in row['page_title']:
                all_types[i] = 'a7'
        elif all_types[i] == 'a7k' and str(row['blocking_key']) == 'sony':
            all_types[i] = 'a7'
        elif re.findall(r'\d+m$', all_types[i]):
            all_types[i] = all_types[i][0:-1]
        elif 'rx100' in all_types[i]:
            if all_types[i].endswith('iii'):
                all_types[i] = all_types[i][0:-3]
            elif all_types[i].endswith('ii'):
                all_types[i] = all_types[i][0:-2]
            elif all_types[i].endswith('m3'):
                all_types[i] = all_types[i][0:-2]
            elif all_types[i].endswith('m2'):
                all_types[i] = all_types[i][0:-2]   
    return ';'.join(all_types)
    
    
    
        

In [786]:
import swifter
structured_data['type'], structured_data['version'] = zip(*structured_data.swifter.apply(lambda row: extract_type(row), axis=1))
structured_data['type_number'] = structured_data.swifter.apply(lambda row: extract_type_number(row), axis=1)
structured_data['type'] = structured_data.swifter.apply(lambda row: some_extra_rule(row), axis=1)


In [695]:
print(extract_type(structured_data.loc['www.ebay.com//53578']))
structured_data.loc['www.ebay.com//53578']['page_title']

('6', '2')


'canon eos rebel ii camera bag and w ef 35 80 mm 1 4 5 6 zoom lens | ebay'

In [375]:
not re.findall(r'\d+', 'coolpix')

True

In [271]:
print(extract_type(structured_data.loc['www.ebay.com//58630']))
structured_data.loc['www.ebay.com//58630']

('60d', -1)


source                                                      www.ebay.com
spec_number                                                        58630
page_title             usa canon eos 60d 18mp canon 50mm 3 lens 8gb d...
blocking_key                                                      cannon
type                                                           50mm3;60d
camera_brand                                                   canon;eos
camera_model                                 canon eos;eos 60d;3;slr;60d
camera_color                                                     unknown
camera_resolution                                                    0.6
camera_sensitivity                                                     0
camera_stablization                                              unknown
camera_battery                                                   unknown
camera_mpn                                                      4460B003
camera_title           USA Canon EOS 60D 18MP Canon

In [320]:
structured_data[structured_data['version'] != '']

,source,spec_number,page_title,blocking_key,type,camera_brand,camera_model,camera_color,camera_resolution,camera_sensitivity,camera_stablization,camera_battery,camera_mpn,camera_title,type_number,version
spec_id,,,,,,,,,,,,,,,,
www.ukdigitalcameras.co.uk//165,www.ukdigitalcameras.co.uk,165,sony cybershot dsc-rx100 iii digital camera (b...,sony,dscrx100,sony,digital camera;dsc-rx100;cybershot,Black,0.670000,0.000000,unknown,unknown,DSCRX100M3.CEH,Sony Cybershot DSC-RX100 III Digital Camera (B...,100,3
www.ukdigitalcameras.co.uk//158,www.ukdigitalcameras.co.uk,158,sony cybershot dsc-rx100 ii digital camera (bl...,sony,dscrx100,sony,digital camera;dsc-rx100 ii;dsc-rx100;cybershot,Black,0.673333,0.000000,unknown,unknown,DSCRX100M2CDN.YG,Sony Cybershot DSC-RX100 II Digital Camera (Bl...,100,2
www.ukdigitalcameras.co.uk//144,www.ukdigitalcameras.co.uk,144,canon powershot g1x mark ii (black) | uk digit...,cannon,g1x,canon,powershot g1x,Black,0.426667,0.000000,unknown,unknown,9167B012AA,Canon PowerShot G1X Mark II (Black) | UK Digit...,1,2
www.wexphotographic.com//152,www.wexphotographic.com,152,canon powershot g1 x mark ii digital camera (9...,cannon,9167b012aa;g1,canon,digital camera;x;g1 x;g1,Black,0.426667,0.250000,unknown,NB-12L,unknown,Canon PowerShot G1 X Mark II Digital Camera (9...,9167012;1,2
www.wexphotographic.com//211,www.wexphotographic.com,211,canon powershot g1 x mark ii digital camera pr...,cannon,g1,canon,digital camera;x;g1 x;g1,Black,0.426667,0.250000,unknown,NB-12L,unknown,Canon PowerShot G1 X Mark II Digital Camera Pr...,1,2
www.wexphotographic.com//600,www.wexphotographic.com,600,sony cyber-shot rx100 ii digital camera - blac...,sony,dscrx100m2cdn.yg;rx100,sony,digital camera;cyber-shot,Black,0.673333,0.250000,unknown,NP-BX1,unknown,Sony Cyber-Shot RX100 II Digital Camera - Blac...,1002;100,2
www.wexphotographic.com//537,www.wexphotographic.com,537,sony cyber-shot rx100 iii digital camera (dscr...,sony,dscrx100m3.ceh;rx100,sony,rx100 iii;digital camera;cyber-shot,Black,0.670000,0.250000,Yes,NP-BX1,unknown,Sony Cyber-Shot RX100 III Digital Camera (DSCR...,1003;100,3
www.canon-europe.com//10,www.canon-europe.com,10,canon powershot g1 x mark ii - powershot and i...,cannon,g1,canon,g1;x;g1 x,unknown,0.000000,0.250000,"Yes (lens shift-type), approx. 3.5-stop¹. Inte...",Rechargeable Li-ion Battery NB-12L (NB-12L bat...,unknown,Canon PowerShot G1 X Mark II - PowerShot and I...,1,2
www.canon-europe.com//164,www.canon-europe.com,164,canon digital ixus ii - powershot and ixus dig...,cannon,ii,canon,NaN,unknown,0.000000,0.007812,unknown,unknown,unknown,Canon Digital IXUS II - PowerShot and IXUS dig...,,2


In [321]:
structured_data.loc['www.alibaba.com//30151']['page_title']

'camera with ip address ds-2cd2632f-is hikvision ir network camera - buy camera with ip address,hikvision cctv camera,hikvision 3mp product on alibaba.com'

In [281]:
structured_data.loc['www.ebay.com//43237']

source                                                      www.ebay.com
spec_number                                                        43237
page_title             perfect waterproof 2 inch tft lcd 8mp digital ...
blocking_key                                                         NaN
type                                                                   2
camera_brand                                           unbranded/generic
camera_model                                                       lcd;2
camera_color                                                     unknown
camera_resolution                                               0.266667
camera_sensitivity                                                     0
camera_stablization                                              unknown
camera_battery                                                   unknown
camera_mpn                                                       unknown
camera_title           Perfect Waterproof 2 inch TF

In [451]:
structured_data.to_csv('total_with_key_type.csv')

In [ ]:
structured_data['type'] = structured_data['type'].fillna('') 

In [ ]:
len(structured_data[pd.isnull(structured_data['type'])])

In [ ]:
import swifter
import re
import dask.dataframe as dd
import multiprocessing
import pandas as pd
import numpy as np
from dask.multiprocessing import get
import editdistance

# stopped_words = set(['digital camera', 'digital', 'camera'])
ddata = dd.from_pandas(candidate_pairs, npartitions=2*multiprocessing.cpu_count())
# ddata = dd.from_pandas(candidate_pairs[0:4], npartitions=2)
def apply_df(df):
#     structured_data = pd.read_csv('total_normalized.csv').set_index('camera_ids')
    print (len(df))
    return df.apply(lambda row: decide_keep(row, structured_data), axis=1)

def decide_keep(row, structured_data):
    stopped_words = set(['digital camera', 'camera']) | all_brands

    llll = structured_data.loc[row['left_spec_id']]
    rrrr = structured_data.loc[row['right_spec_id']]

    lset = set(str(llll['type']).split(';'))
    rset = set(str(rrrr['type']).split(';'))

    max_len = max(len(str(llll['type'])), len(str(rrrr['type'])))
    camera_type_edit_dis=(editdistance.eval(''.join(sorted(lset)), ''.join(sorted(rset))) / float(max_len))

    if len(lset) == 0 and len(rset) == 0:
        camera_type_jaccard_dis=(0.0)
        camera_type_overlap_dis=(0.0)
    else:
        camera_type_jaccard_dis=(1.0 - len(lset&rset) / float(len(lset|rset)))
        camera_type_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

    lset = set(re.findall(r"\d+\.?\d*",str(llll['type'])))
    rset = set(re.findall(r"\d+\.?\d*",str(rrrr['type'])))

    if len(lset) == 0 and len(rset) == 0:
        camera_type_numeric_jaccard_dis=(0.0)
        camera_type_numeric_overlap_dis=(0.0)
    else:
        camera_type_numeric_jaccard_dis=(1-len(lset&rset) / float(len(lset|rset)))
        camera_type_numeric_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

    lnumtitle = re.findall(r"\d+",str(llll['page_title']))
    rnumtitle = re.findall(r"\d+",str(rrrr['page_title']))

    lset = set(lnumtitle)
    rset = set(rnumtitle)

    if len(lset) == 0 and len(rset) == 0:
        camera_title_numeric_jaccard_dis=(0.0)
        camera_title_numeric_edit_dis=(0.0)
    else:
        camera_title_numeric_jaccard_dis=(1.0 - len(lset&rset) / float(len(lset|rset)))
        max_len = max(len(lnumtitle), len(rnumtitle))
        camera_title_numeric_edit_dis=(editdistance.eval(lnumtitle, rnumtitle) / float(max_len))

    ltitle = str(llll['page_title']).split(' ')
    rtitle = str(rrrr['page_title']).split(' ')
    lset = set(ltitle)
    rset = set(rtitle)

    if len(lset) == 0 and len(rset) == 0:
        camera_title_jaccard_dis=(0.0)
        camera_title_overlap_dis=(0.0)
        camera_title_edit_dis=(0.0)
    else:
        camera_title_jaccard_dis=(1.0-len(lset&rset) / float(len(lset|rset)))
        camera_title_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))
        max_len = max(len(ltitle), len(rtitle))
        camera_title_edit_dis=(editdistance.eval(ltitle, rtitle) / float(max_len))
    overlap_freqs = []
    for w in lset&rset:
        if w in word_tfidf:
            overlap_freqs.append(str(word_tfidf[w]))
    camera_title_overlap_freq=(';'.join(overlap_freqs))
    unmatch_freqs = []
    for w in (lset|rset)-(lset&rset):
        if w in word_tfidf:
            unmatch_freqs.append(str(word_tfidf[w]))
    camera_title_unmatch_freq=(';'.join(unmatch_freqs))
        
    return [camera_type_jaccard_dis, camera_type_overlap_dis,
            camera_type_edit_dis, camera_type_numeric_jaccard_dis,
            camera_type_numeric_overlap_dis,camera_title_jaccard_dis,
            camera_title_overlap_dis,camera_title_edit_dis,
            camera_title_numeric_jaccard_dis,camera_title_numeric_edit_dis,
            camera_title_overlap_freq,camera_title_unmatch_freq]

print ('begin')
# column = {'camera_type_jaccard_dis':float,'camera_type_numeric_jaccard_dis':float,'camera_title_jaccard_dis':float,'camera_type_overlap_dis':float,'camera_type_numeric_overlap_dis':float,'camera_title_overlap_dis':float,'camera_title_overlap_freq':str}
# res = ddata.apply(lambda row: decide_keep(row, structured_data), axis=1, meta=pd.Series(dtype=float,name='x')).compute(scheduler='processes')
res = ddata.map_partitions(lambda df: apply_df(df), meta=pd.Series(name='x')).compute(scheduler='processes') 
# candidate_pairs['keep'] = candidate_pairs.swifter.apply(lambda row: decide_keep(), axis=1)


In [ ]:
pairs_df['camera_type_jaccard_dis'] = camera_type_jaccard_dis
    pairs_df['camera_type_overlap_dis'] = camera_type_overlap_dis
    pairs_df['camera_type_edit_dis'] = camera_type_edit_dis
    pairs_df['camera_type_numeric_jaccard_dis'] = camera_type_numeric_jaccard_dis
    pairs_df['camera_type_numeric_overlap_dis'] = camera_type_numeric_overlap_dis
    pairs_df['camera_title_jaccard_dis'] = camera_title_jaccard_dis
    pairs_df['camera_title_overlap_dis'] = camera_title_overlap_dis
    pairs_df['camera_title_edit_dis'] = camera_title_edit_dis
    pairs_df['camera_title_numeric_jaccard_dis'] = camera_title_numeric_jaccard_dis
    pairs_df['camera_title_numeric_edit_dis'] = camera_title_numeric_edit_dis
    pairs_df['camera_title_overlap_freq'] = camera_title_overlap_freq
    pairs_df['camera_title_unmatch_freq'] = camera_title_unmatch_freq

In [ ]:
res[10000]

In [ ]:

candidate_pairs['camera_type_jaccard_dis'] = [x[0] for x in res]
candidate_pairs['camera_type_overlap_dis'] = [x[1] for x in res]
candidate_pairs['camera_type_edit_dis'] = [x[2] for x in res]
candidate_pairs['camera_type_numeric_jaccard_dis'] = [x[3] for x in res]
candidate_pairs['camera_type_numeric_overlap_dis'] = [x[4] for x in res]
candidate_pairs['camera_title_jaccard_dis'] = [x[5] for x in res]
candidate_pairs['camera_title_overlap_dis'] = [x[6] for x in res]
candidate_pairs['camera_title_edit_dis'] = [x[7] for x in res]
candidate_pairs['camera_title_numeric_jaccard_dis'] = [x[8] for x in res]
candidate_pairs['camera_title_numeric_edit_dis'] = [x[9] for x in res]
candidate_pairs['camera_title_overlap_freq'] = [x[10] for x in res]
candidate_pairs['camera_title_unmatch_freq'] = [x[11] for x in res]


In [ ]:
candidate_pairs.loc[0:10]

In [ ]:
candidate_pairs.to_csv('candidates_with_distance.csv', index=False)

In [ ]:
candidate_pairs = pd.read_csv('candidates_with_distance.csv')

In [ ]:
candidate_pairs['keep'] = res

In [ ]:
candidate_pairs[candidate_pairs['keep'] == 1].to_csv('candidates_2.csv')

# Put them together

In [ ]:
candidate_pairs['camera_title_overlap_freq'] = candidate_pairs['camera_title_overlap_freq'].fillna('')

In [ ]:
# distances_data = list(candidate_pairs.iloc[:, 4:12].values)
# # freqs_data = list(candidate_pairs.iloc[:, 11:12].values)
# others_data = list(candidate_pairs.iloc[:, 4:5].values)

# data_loader = torch.utils.data.DataLoader(
#         ProductsDataset(distances_data, others_data), batch_size=1024, shuffle=False)

# distances_model.eval()
# freqs_model.eval()
output_model.eval()
results = []
matches = 0
start = time.time()
for batch_idx, (distances, targets) in enumerate(data_loader):
    distance = Variable(distances)
#     target = Variable(targets)
    freq = Variable(freqs)

#     distance_embedding = distances_model(distance) 
#     freq_embedding = freqs_model(freq)
    estimates = output_model(distance)
    
    for i in range(estimates.shape[0]):
        if estimates[i].item() >= 0.5:
            results.append(1)
            matches += 1
        else:
            results.append(0)
            
    end = time.time()
    print ('Testing: Iteration {}, Time: {}s, MatchNum: {}, CurrentNum: {}'.format(e, end-start, matches, len(results)))
    start = time.time()
# print ('Testing: Iteration {}'.format(e, np.mean(mse_errors), np.mean(precisions), np.mean(recalls)))


In [ ]:
len(results)

In [ ]:
candidate_pairs['predict'] = results

In [ ]:
candidate_pairs[candidate_pairs['predict'] == 1][['left_spec_id', 'right_spec_id']].to_csv('submit_v7.csv', index=False)


In [ ]:
candidate_pairs[candidate_pairs['predict'] == 1].to_csv('candidates_positive_with_distance.csv', index=False)

In [ ]:
len(candidate_pairs[candidate_pairs['predict'] == 0])